In [2]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector

from qiskit_aer.primitives import Estimator as AerEstimator

from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.settings import QiskitNatureSettings

QiskitNatureSettings.use_pauli_sum_op = False

## Exercise

We will continue last week's exercise and work with the Hubbard model again,
$$H = - t \sum_{i,\sigma} a^\dagger_{i,\sigma} a_{i+1,\sigma} + a^\dagger_{i+1,\sigma} a_{i,\sigma} + U \sum_i n_{i, \uparrow} n_{i, \downarrow} \ .$$

We again use a qubit ordering of the form $\bigotimes_i | q_{i,\uparrow} \rangle \bigotimes_i | q_{i,\downarrow} \rangle$ (like last week).

The goal of this exercise is to explore the influence of statistical (shot) noise on the convergence of the VQE.
Specifically, for a Hubbard model at half filling (i.e., half of the sites are occupied with a spin-up electron) with $t = U = 1$ for both $N=2$ and $N=4$ sites.

- Implement the cost function using simulation shots instead of statevectors
- This time, track the value of the cost function throughout the minimization, i.e. store the cost function value at intermediate iterations
- Plot the cost function value as a function of the iterations for different numbers of shots $N_{\rm shots} \in \{ 100, 500, 1000 \}
- check how the accuracy changes with shots
- Using this cost function, perform the minimization using `scipy.optimization.minimize`
    - using `method="COBYLA"`
    - Provide an initial guess for the parameter values
    - Pass the necessary additional arguments of the cost function as `args=(...)` to `minimize`.
- Repeat this for $L \in \{ 0, 1, 2, 3, 4 \}$ circuit layers of the ansatz ($L=0$ meaning you only have the initial state, including the initial layer of variational gates to prepare the non-interacting ground state of the Hubbard Hamiltonian)
- For each $L$ and $N$, store the final result of the minimization (VQE), i.e.
    - final value of the cost function (the energy)
    - final parameter values
    - the infidelity with the exact ground state
- **Print and plot** the results as a function of $L$, specifically
    - the relative energy error $|(E_{\rm vqe} - E_{\rm exact}) / E_{\rm exact}|$
    - the infidelity with respect to the exact ground state

In [3]:
def sortES(eigVals, eigVecs):
    """
    simple routine to sort eigenvectors given the eigenvalues
    """

    # Zip the associated eigenvalues and eigenvectors together and sort by eigenvalue
    eigSystem = sorted(zip(eigVals, np.transpose(eigVecs)), key=lambda x: x[0])
    eigValsOut = [eVal for eVal, _ in eigSystem]
    eigVecsOut = [eVec for _, eVec in eigSystem]

    return eigValsOut, eigVecsOut

In [4]:
def get_hubbard_hamiltian(num_sites, t, U):
    """Constructs the Hubbard Hamiltonian in the dict format supported by qiskit-nature.
    Returns a list with two elements, the first being a dict containing the hopping terms,
    the second the onsite terms.
    """
    hopping = {}
    onsite = {}
    for n in range(num_sites - 1):
        hopping[f"+_{n + 1} -_{n}"] = - t
        hopping[f"+_{n} -_{n + 1}"] = - t
        hopping[f"+_{n + 1 + num_sites} -_{n + num_sites}"] = - t
        hopping[f"+_{n + num_sites} -_{n + 1 + num_sites}"] = - t

        onsite[f"+_{n} -_{n} +_{n + num_sites} -_{n + num_sites}"] = U
        onsite[f"+_{n + 1} -_{n + 1} +_{n + 1 + num_sites} -_{n + 1 + num_sites}"] = U

    return [hopping, onsite]

In [5]:
def get_init_circ(num_sites):
    """Constructs the initial state for the variational Hamiltonian ansatz for a Hubbard model at half filling.
    First creates the half-filling state (i.e., half the sites are occupied with a spin-up electron).
    Then appends layers of Givens rotations (see https://www.nature.com/articles/s41467-022-33335-4
    or https://arxiv.org/abs/2302.09824) that variationally prepares the ground state of a non-interacting
    Hubbard model (i.e., no onsite potential, U=0).
    """
    param = Parameter("param")
    Ggate = QuantumCircuit(2, name="G")
    Ggate.h(0)
    Ggate.cx(0, 1)
    Ggate.ry((param / 2), 0)
    Ggate.ry((param / 2), 1)
    Ggate.cx(0, 1)
    Ggate.h(0)

    init_circ = QuantumCircuit(2 * num_sites)
    for i in range(int(num_sites / 2)):
        init_circ.x([i, i + num_sites])

    if num_sites == 2:
        theta = ParameterVector("theta", 1)
        for i in range(num_sites - 1):
            init_circ.append(Ggate.to_instruction({param: theta[0]}), [i, i + 1])
            init_circ.append(
                Ggate.to_instruction({param: theta[0]}),
                [i + num_sites, i + 1 + num_sites],
            )
    else:
        theta = ParameterVector("theta", 3)
        for i in range(1, num_sites - 1, 2):
            init_circ.append(Ggate.to_instruction({param: theta[0]}), [i, i + 1])
            init_circ.append(
                Ggate.to_instruction({param: theta[0]}),
                [i + num_sites, i + 1 + num_sites],
            )
        for i in range(0, num_sites - 1, 2):
            init_circ.append(Ggate.to_instruction({param: theta[1]}), [i, i + 1])
            init_circ.append(
                Ggate.to_instruction({param: theta[1]}),
                [i + num_sites, i + 1 + num_sites],
            )
        for i in range(1, num_sites - 1, 2):
            init_circ.append(Ggate.to_instruction({param: theta[2]}), [i, i + 1])
            init_circ.append(
                Ggate.to_instruction({param: theta[2]}),
                [i + num_sites, i + 1 + num_sites],
            )

    return init_circ

In [6]:
def cost_fun(x, ansatz, ops, estimator):

    # At each optimization step, we bind the new parameter values to the ansatz
    # (the parametrized circuit) and compute the energy expectation value

    cost = # compute cost from results

    return cost

## $N=2$ sites

In [ ]:
# Define the system parameters
num_sites = 2
t = 1
U = 1

# get the dictionary of second quantization operators
hubbard_ops = get_hubbard_hamiltian(num_sites, t, U)

# map the second quantization operators to Pauli operators
num_orbitals = 2 * num_sites
ops_mapped = []
for term in hubbard_ops:
    op = FermionicOp(
        term,
        num_spin_orbitals=num_orbitals,
    )
    print(op)

    ops_mapped.append(JordanWignerMapper().map(op))

# get the Hamiltonian matrix
H = sum(ops_mapped)
H_mat = H.to_matrix()

# get the exact ground state and corresponding energy (for verification)
eigVals, eigVecs = sp.linalg.eig(H)
eigVals, eigVecs = sortES(eigVals, eigVecs)
en_exact = np.real(eigVals[0])
gs_exact = eigVecs[0]

print("\nGround state energy exact")
print(np.real(en_exact))

In [ ]:
# do VQE for different numbers of layers
ansatz_operators = [SparsePauliOp(op.paulis) for op in ops_mapped[::-1]]

init_circ = get_init_circ(num_sites)

shot_vals = [10, 100, 500, 1000]

# we will do VQE for different numbers of layers
num_layers = [0, 1, 2, 3, 4]
vqe_results = {}

for shots in shot_vals:
    en_vqe = []
    params_vqe = []
    infid_vqe = []
    iterations_vqe = []

    for reps in num_layers:
        print(f"\n****************************************")
        print(f"*** VQE for {shots} shots, {reps} layers")
        print(f"****************************************", end="\n\n")

        ansatz = 

        obj_fun = lambda x: cost_fun(x, ansatz, H, shots)

        # store all intermediate function evaluations (each iteration), e.g., through a callback

        opt_res = # Do VQE

        print(f"\nFINAL results:\n\n{opt_res}")

        infid_vqe.append(1 - fidelity)
        en_vqe.append(opt_res.fun)
        params_vqe.append(opt_res.x)
        iterations_vqe.append(iterations)


    vqe_results[shots] = {
        "sites": num_sites,
        "t": t,
        "U": U,
        "en_exact": en_exact,
        "layer": num_layers,
        "energy": en_vqe,
        "infidelity": infid_vqe,
        "parameters": params_vqe,
        "iterations": iterations_vqe,
    }

### Plot final results

In [ ]:
ncols = 2
nrows = 3
fig = plt.figure(figsize=(4 * ncols, 4 * nrows))
cmap = plt.get_cmap("tab10")

gs = fig.add_gridspec(nrows, ncols, wspace=0.3)
axs = []
axs.append(fig.add_subplot(gs[0, 0]))
axs.append(fig.add_subplot(gs[0, 1]))
axs.append(fig.add_subplot(gs[1, :]))
axs.append(fig.add_subplot(gs[2, :]))

for shots in shot_vals:
    delta_E = np.abs(
        (en_exact - vqe_results[shots]["energy"])
        / en_exact
    )
    axs[0].plot(vqe_results[shots]["layer"], delta_E, label=f"{shots} shots")
    axs[0].set_xlabel(rf"layer")
    axs[0].set_ylabel(rf"$\Delta E$")
    axs[0].set_yscale("log")

    axs[1].plot(vqe_results[shots]["layer"], vqe_results[shots]["infidelity"])
    axs[1].set_xlabel(rf"layer")
    axs[1].set_ylabel(rf"$\langle \psi_{{\rm init}} | \psi(t) \rangle$")
    axs[1].set_yscale("log")

marker = ["o", "d", "<", ">", "x", "+"]

iterations_vqe = vqe_results[shot_vals[-1]]["iterations"]
for layer in range(len(iterations_vqe)):
    axs[2].plot(
        iterations_vqe[layer].keys(),
        iterations_vqe[layer].values(),
        label=f"L={num_layers[layer]}, {shot_vals[-1]} shots",
        color=cmap(layer),
    )
    axs[2].plot(
        list(iterations_vqe[layer].keys())[0],
        list(iterations_vqe[layer].values())[0],
        color=cmap(layer),
        marker=marker[layer],
        markersize=10,
    )
    axs[2].set_xlabel(rf"iteration")
    axs[2].set_ylabel(rf"$\langle H \rangle$")
    # axs[2].set_yscale("log")

layer_idx = 2
for i, shots in enumerate(shot_vals):
    axs[3].plot(
        vqe_results[shots]["iterations"][layer_idx].keys(),
        vqe_results[shots]["iterations"][layer_idx].values(),
        label=f"L={num_layers[layer_idx]}, {shots} shots",
        color=cmap(i),
    )
    axs[3].plot(
        list(vqe_results[shots]["iterations"][layer_idx].keys())[0],
        list(vqe_results[shots]["iterations"][layer_idx].values())[0],
        color=cmap(i),
        marker=marker[i],
        markersize=10,
    )
    axs[3].set_xlabel(rf"iteration")
    axs[3].set_ylabel(rf"$\langle H \rangle$")
    # axs[2].set_yscale("log")

axs[2].plot([-1, 1000], [en_exact, en_exact], ls="--", color="grey")
axs[3].plot([-1, 1000], [en_exact, en_exact], ls="--", color="grey")

axs[2].set_xlim(-2, 80)
axs[3].set_xlim(-2, 56)

axs[0].legend()
axs[2].legend()
axs[3].legend()

plt.show()